In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["WANDB_SILENT"] = "true"

In [ ]:
import tensorflow as tf
import os
import glob
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint

Define Dataset Loading Function

In [ ]:
def load_dataset(tfrecord_pattern, batch_size=64, shuffle=True):
    # Get list of all TFRecord files in the directory
    files = glob.glob(tfrecord_pattern)

    # Define your feature description for parsing TFRecords
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),  # Serialized as bytes
        'label': tf.io.FixedLenFeature([4], tf.int64)   # Assuming labels are one-hot encoded
    }

    # Function to parse the TFRecord examples
    def _parse_function(proto):
        parsed_example = tf.io.parse_single_example(proto, feature_description)
        
        # Decode image bytes to proper image tensor
        image = tf.io.decode_jpeg(parsed_example['image'], channels=3)  # Adjust this to decode the image properly
        image = tf.image.resize(image, (224, 224))  # Resize to match input shape
        label = parsed_example['label']
        return image, label

    # Define the dataset
    raw_dataset = tf.data.TFRecordDataset(files)
    dataset = raw_dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)

    # Optionally shuffle the data and batch it
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1000)

    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset

# Load the datasets
train_dataset = load_dataset("/kaggle/input/oct-dataset-tfrecord/train/train_chunk_*.tfrecord", batch_size=64)
val_dataset = load_dataset("/kaggle/input/oct-dataset-tfrecord/val/val_chunk_*.tfrecord", batch_size=64, shuffle=False)
test_dataset = load_dataset("/kaggle/input/oct-dataset-tfrecord/test/test_chunk_*.tfrecord", batch_size=64, shuffle=False)

Set up Mirrored Strategy

In [ ]:
import tensorflow as tf

# Initialize the strategy for multi-GPU training
strategy = tf.distribute.MirroredStrategy()

print(f"Number of devices: {strategy.num_replicas_in_sync}")

Define the Model and Wrap in Strategy Scope

In [ ]:
with strategy.scope():
    from tensorflow.keras.applications import EfficientNetB7
    from tensorflow.keras import layers, models

    # Define the model
    base_model = EfficientNetB7(
        include_top=False,
        weights='imagenet',  # Use pre-trained weights
        input_shape=(224, 224, 3)
    )

    # Freeze the pre-trained layers
    base_model.trainable = False

    # Add custom head for classification
    head_model = models.Sequential([
        layers.GlobalAveragePooling2D(),
        layers.Dense(4, activation='softmax')  # Output 4 classes
    ])

    # Combine the base and custom head
    final_model = models.Sequential([
        base_model,
        head_model
    ])

    # Compile the model
    final_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

Set up Callbacks for Checkpointing

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_path = "/kaggle/working/ckpt_{epoch:02d}.weights.h5"
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_best_only=True,  # Save only the best model
    monitor='val_loss',
    mode='min',
    verbose=1
)

Train the Model

In [ ]:
final_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,  # Start with 50 epochs
    callbacks=[checkpoint_callback]
)

Adding Test Dataset

In [ ]:
test_loss, test_accuracy = final_model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")